In [ ]:
import os

import numpy as np
from sklearn.linear_model import LogisticRegression

from utils import load_hidden_representations_from_hdf5, read_templates_from_file

----

In [ ]:
# params
log_dir = "/logfiles"
model = "bigscience-T0_3B"
module = "encoder"
task = "rte"

In [ ]:
# assertions
assert model == "bigscience-T0_3B"
assert module == "encoder"
assert task == "rte"

## Prompts

In [ ]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}/all.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_prompt.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_target_yes_no.csv")
display(df)

## Train linear classifier

In [ ]:
def unison_shuffled_copies(a, b):
    # from: https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [ ]:
use_pattern = [
    "gpt_3_yes_no_with_targets",
    "mnli_crowdsource_with_targets",
    "based_on_previous_passage_with_targets",
    "infer_with_targets",
    # "follow_with_targets",
    # "imply_with_targets",
    # "guaranteed_with_targets",
    # "justified_with_targets",
    # "must_be_true_with_targets",
    # "should_assume_with_targets",
    # "gpt_3_true_false_with_targets",
    # "gpt_3_cat_dog_with_targets",
    # "gpt_3_cat_dog_with_targets_yes_no",
    # "gpt_3_yes_no_without_targets",
    # "words_appear_with_targets",
    # "similar_words_with_targets",
    # "start_with_the_with_targets",
    # "same_meaning_with_targets",
    # "paraphrase_with_targets",
    # "paraphrase_r_with_targets",
    # "summarize_with_targets",
    # "inflection_with_targets",
    # "null_pattern_with_targets",
    # "null_pattern_r_with_targets",
    # "null_pattern_without_targets",
    # "null_pattern_r_without_targets",
    # "premise_only_with_targets",
    # "premise_only_without_targets",
    # "hypothesis_only_with_targets",
    # "hypothesis_only_without_targets",
]

In [ ]:
# for layer in range(0, 3):
for layer in range(0, 25):
# for layer in range(24, 25):
    
    print("layer:", layer)
    file_names, prompt_names = [], []

    for _, row in df.iterrows():
        if row['name'] in use_pattern:
            # file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_layer{layer}_avg.hdf5",)
            file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_layer{layer}_avg-nopad.hdf5",)
            prompt_names.append(row['name'])

    # load hidden representations from hdf5 file
    representations = None
    classes = []
    n_sequences = 0

    for idx, file_name in enumerate(file_names):
        hidden_representations = load_hidden_representations_from_hdf5(os.path.join(log_dir, file_name))
        # take only first 100 samples
        hidden_representations = hidden_representations[:100, :]
        # print(hidden_representations.shape)
        n_sequences = hidden_representations.shape[0]

        if representations is None:
            representations = hidden_representations
        else:
            representations = np.concatenate((representations, hidden_representations), axis=0)

        # use pattern id as label
        # classes += n_sequences * [idx] # assign representations to classes
        # use sample id as label
        classes += list(range(n_sequences)) # assign representations to classes
        
    classes = np.asarray(classes)
    # print(classes)

    X, y = representations, classes
    # shuffle representations and classes
    # X, y = unison_shuffled_coSpies(X, y)
    print(X.shape, y.shape)
    # print(y)

    # train linear classifier
    # multi_class='multinomial' uses a CE loss
    # print('classifying between:', prompt_names)
    print('classifying input ids for prompts:', prompt_names)
    clf = LogisticRegression(random_state=0, max_iter=1000, multi_class='multinomial').fit(X, y)
    
    print(f'layer={layer}; accuracy on training data: ', clf.score(X, y))
    print('\n')



In [ ]:
# for layer in range(0, 3):
for layer in range(0, 25):
# for layer in range(24, 25):
    
    print("layer:", layer)
    file_names, prompt_names = [], []

    for _, row in df.iterrows():
        if row['name'] in use_pattern:
            # file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_layer{layer}_avg.hdf5",)
            file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_layer{layer}_avg-nopad.hdf5",)
            prompt_names.append(row['name'])

    # load hidden representations from hdf5 file
    representations = None
    classes = []
    n_sequences = 0

    for idx, file_name in enumerate(file_names):
        hidden_representations = load_hidden_representations_from_hdf5(os.path.join(log_dir, file_name))
        # take only first 100 samples
        hidden_representations = hidden_representations[:100, :]
        # print(hidden_representations.shape)
        n_sequences = hidden_representations.shape[0]

        if representations is None:
            representations = hidden_representations
        else:
            representations = np.concatenate((representations, hidden_representations), axis=0)

        # use pattern id as label
        classes += n_sequences * [idx] # assign representations to classes
        # use sample id as label
        # classes += list(range(n_sequences)) # assign representations to classes
        
    classes = np.asarray(classes)
    # print(classes)

    X, y = representations, classes
    # shuffle representations and classes
    # X, y = unison_shuffled_coSpies(X, y)
    print(X.shape, y.shape)
    # print(y)

    # train linear classifier
    # multi_class='multinomial' uses a CE loss
    print('classifying between:', prompt_names)
    # print('classifying input ids for prompts:', prompt_names)
    clf = LogisticRegression(random_state=0, max_iter=1000, multi_class='multinomial').fit(X, y)
    
    print(f'layer={layer}; accuracy on training data: ', clf.score(X, y))
    print('\n')

